# Manual Regard on Elastic Mill Spring

In [ ]:
# only for Google Colab
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/requirements.txt"
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/elastic-mill-spring/Auffederung.png"
!pip install -r "requirements.txt"

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

import pyroll.basic as pr
import pyroll.export as pe
from scipy.optimize import root_scalar

## Modeling the Elastic Mill Spring

The mill stand is usually treated as rigid, which is not the reality. To be more accurate, the mill stand should be treated as a system of springs, which includes the mill stand itself, the parts for the bearing of the rolls and the rolls themselves. Their principal reaction can be seen in the following diagram for defined mill. But the amplitude of this phenomenon is different from mill stand to mill stand and needs to be specified for each of those. The mills elastic response is quite complex, so we usually apply simplified models with empirically fitted coefficients.

![](Auffederung.png)

For this example case the system of springs is simplified as a single spring constant $C_\mathrm{S}$ in dependence on the rolling force $F_\mathrm{R}$ (for example through previous calculations) being available the real roll gap $s_1$ can be, according to Weber, calculated by:
$$s_1 = s_0 + \frac{F_\mathrm{R}}{C_\mathrm{S}}$$

Or inverted:
$$ F_\mathrm{S} = C_\mathrm{S} \cdot (s_1 - s_0) $$

## Definition of the Rolling Process Using PyRolL

We define an initial workpiece.

In [ ]:
in_profile = pr.Profile.box(
    height=4e-3,
    width=50e-3,
    corner_radius=0,
    temperature=25 + 273.15,
    strain=0,
    material="C45",
    density=7.5e3,
    specific_heat_capacity=690,
)
in_profile

The roll pass is given by a factory function that has the *current* roll gap as parameter. 

In [ ]:
def roll_pass(roll_gap):
    return pr.RollPass(
        label="Flach Sack I",
        roll=pr.Roll(
            groove=pr.FlatGroove(
                usable_width=250e-3,
            ),
            nominal_radius=150e-3,
        ),
        gap=roll_gap,
        velocity=0.1,
        coulomb_friction_coefficient=0.2,
    )

The nominal roll gap is the roll gap set on the mill without any load ($s_0$).

In [ ]:
nominal_roll_gap = 2e-3

## Calculating the Roll Force's Dependence on the Roll Gap

We create a data frame and fill it with the roll gaps where the simulations shall be conducted, from nominal roll gap up to the initial height of the workpiece (zero draught). Zero draught will not be simulated as this would mean undefined results due to division by zero in some models (`endpoint=False`).

In [ ]:
df = pd.DataFrame(
    {"roll_gap": np.linspace(nominal_roll_gap, in_profile.height, 100, endpoint=False)}
)
df

Then we simulate rolling at all these roll gaps to obtain the respective roll forces. This may take a while depending on the used models. Some warnings due to failed convergence at the boundaries of this interval are to be expected.

In [ ]:
for i in df.index:
    roll_gap = df["roll_gap"][i]
    print(roll_gap)
    rp = roll_pass(roll_gap)
    rp.solve(in_profile)
    df.loc[i, "roll_force"] = rp.roll_force

The data frame now stores the obtained roll forces.

In [ ]:
df

We plot the roll force over the roll gap.

In [ ]:
px.line(df, x="roll_gap", y="roll_force")

### Calculating the Mill Spring

As stated above, we approximate the mill spring by a linear spring model. We define the spring force depending on the roll gap widening (`delta_s`).

In [ ]:
def mill_spring_force(delta_s):
    """Characteristic curve of mill spring."""
    return 704.1e7 * delta_s

We calculate the respective spring force for all of the previously used roll gaps and store the results in the same data frame.

In [ ]:
df["spring_force"] = mill_spring_force(df.roll_gap - nominal_roll_gap)
df

As the mill spring curve is much steaper, axis scaling is unfortunate.

In [ ]:
px.line(df, x="roll_gap", y=["spring_force", "roll_force"])

We mask the spring forces higher than the maximum roll force by replacing them with `NA` (not available).

In [ ]:
df["spring_force_masked"] = df.spring_force
df.loc[df.spring_force > df.roll_force.max(), "spring_force_masked"] = pd.NA
df

Plotting those masked values, the plot looks much nicer.

In [ ]:
px.line(df, x="roll_gap", y=["spring_force_masked", "roll_force"])

We may calculate the intersection numerically using the `root_scalar` function of scipy. We interpolate our simulation results. For the mill spring, we directly use the analytical expression.

In [ ]:
root_scalar(
    lambda s: np.interp(s, df.roll_gap, df.roll_force)
    - mill_spring_force(s - nominal_roll_gap),
    bracket=[nominal_roll_gap, in_profile.height],
)